# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/987.6 kB ? eta -:--:--
     - ------------------------------------- 30.7/987.6 kB 1.3 MB/s eta 0:00:01
     - ----------------------------------- 41.0/987.6 kB 487.6 kB/s eta 0:00:02
     --------------- ---------------------- 399.4/987.6 kB 3.5 MB/s eta 0:00:01
     -------------------- ----------------- 522.2/987.6 kB 3.6 MB/s eta 0:00:01
     -------------------------- ----------- 686.1/987.6 kB 3.3 MB/s eta 0:00:01
     -------------------------------------  983.0/987.6 kB 3.7 MB/s eta 0:00:01
     -------------------------------------- 987.6/987.6 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
     -- ------------------------------------- 0.2/2.1 MB 9.0 MB/s eta 0:00:01
     -- ------------------------------------- 0.2/2.1 MB 9.0 MB/s eta 0:00:01
     --- ------------------------------------ 0.2/2.1 MB 2.5 MB/s eta 0:00:01
 

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [57]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [58]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


biology_template = """你是一位经验丰富的生物学教授。
你擅长用直观的方式回答关于生物学的问题。
当你面对一个未知的问题时，你会进行研究，直到找到答案。

这是一个问题：
{input}"""

computer_science_template = """你是一位资深的计算机科学家。
你对编程和计算机系统有深入的理解。
遇到技术难题时，你会逐步分析问题，直到找到解决方案。

这是一个问题：
{input}"""

chinese_literature_template = """你是一位汉语言文学的专家。
你对中国的文学作品和语言文化有深入的了解。
面对相关问题，你能提供详尽的背景信息和精准的解读。

这是一个问题：
{input}"""

In [59]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机科学问题",
        "prompt_template": computer_science_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_literature_template,
    }    
]

In [60]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", base_url="https://api.xiaoai.plus/v1", api_key="sk-20kg2ng944ZYhmdr87Db79E9323e48DaA1470c40B4B5439c")

In [61]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [62]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [63]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [64]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [65]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物学问题', '计算机: 适用于回答计算机科学问题', '汉语言文学: 适用于回答汉语言文学问题']


In [66]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机科学问题
汉语言文学: 适用于回答汉语言文学问题


In [67]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [68]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机科学问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

In [69]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [70]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？'}
> Finished chain.
{'input': '黑体辐射是什么？', 'text': '\n\n黑体辐射是指一个处于热平衡状态的理想物体，它会发出电磁辐射，其强度和频率分布与温度有关。根据普朗克定律和斯特藩-玻尔兹曼定律，黑体辐射的强度和频率分布可以被精确计算。黑体辐射在物理学中具有重要的意义，它可以用来解释许多自然现象，例如太阳的光谱分布和宇宙微波背景辐射。'}


In [71]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要找出大于40的第一个质数。一个简单的方法是从41开始，依次判断每个数是否是质数。通过这种方法，我们可以得知大于40的第一个质数是41。\n\n接着，我们需要找出41+1=42能被3整除的数。一个简单的方法是通过试除法，依次除以3，直到余数为0为止。通过这种方法，我们可以得知42除以3得到的余数为0，因此41+1=42能被3整除。\n\n因此，大于40的第一个质数是41，使得这个质数加一能被3整除。'}


In [17]:
router_chain.verbose = True

In [72]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是一种非常密集的天体，它的引力非常强大，以至于连光也无法逃逸。它的存在可以通过观测它造成的物质运动和辐射来推断，因为我们无法直接看到它。在宇宙中，黑洞通常是由大质量恒星坍缩而成的，它们的引力场非常强大，甚至可以吸引和撕裂附近的物质。黑洞是宇宙中最神秘和最奇特的天体之一，也是物理学研究的重要课题。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [73]:
print(chain.invoke("covid是什么？?"))



> Entering new MultiPromptChain chain...
None: {'input': 'covid是什么？?'}
> Finished chain.
{'input': 'covid是什么？?', 'history': '', 'text': ' COVID是一种传染病，也被称为新冠病毒。它最早于2019年在中国武汉被发现，随后迅速蔓延到全世界。这种病毒主要通过呼吸道飞沫传播，患者可能会出现发热、咳嗽、乏力等症状。目前，全世界都在努力抗击这种疾病，包括研发疫苗和采取措施控制传播。'}


In [75]:
print(
    chain.invoke(
        "可以介绍下计算机网络的应用场景吗？"
    )
)



> Entering new MultiPromptChain chain...
计算机: {'input': '可以介绍下计算机网络的应用场景吗？'}
> Finished chain.
{'input': '可以介绍下计算机网络的应用场景吗？', 'text': '\n\n计算机网络的应用场景非常广泛，可以说几乎涵盖了现代社会生活的方方面面。以下列举几个典型的应用场景：\n\n1. 互联网：互联网是计算机网络技术最广泛的应用，它连接了全世界的计算机和终端设备，使人们可以通过浏览器、手机等终端设备在网络上进行信息交流、资源共享、网上购物等各种活动。\n\n2. 移动通信网络：移动通信网络是一种无线网络，它使得人们可以通过手机、平板电脑等移动终端设备进行语音通话、短信、视频通话、移动支付等各种服务。\n\n3. 企业内部网络：企业内部网络是指连接企业内部各种计算机和设备的局域网，它可以实现企业内'}


In [77]:
print(chain.invoke("介绍下红楼梦的创作背景？"))



> Entering new MultiPromptChain chain...
汉语言文学: {'input': '请介绍一下《红楼梦》的创作背景'}
> Finished chain.
{'input': '请介绍一下《红楼梦》的创作背景', 'text': '和主要内容。\n\n《红楼梦》是中国古典文学四大名著之一，也被称为《石头记》。它由清代作家曹雪芹所著，共120回。曹雪芹在创作《红楼梦》时，受到了社会政治动荡和个人经历的影响。他曾在康熙皇帝的侄女四美公主家当过书童，也曾参与过史学编纂工作，因此对当时的政治和社会有着深刻的了解。\n\n《红楼梦》主要讲述了贾、史、王、薛四大家族之间的婚姻联姻和贾宝玉与林黛玉的爱情故事。整部作品以贾府为中心，通过对贾府内'}
